# Исследование надёжности заёмщиков

### Влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок?

<div class="alert alert-info" role="alert">
    <p class="mb-0">Проект делаю на локальном Jupyter Lab. Поэтому набор данных загружаю из <code>./datasets/data.csv</code></p>
</div>

In [ ]:
import pandas as pd
import statistics
import seaborn as sns

In [ ]:
# import nltk
# nltk.download("stopwords")

In [ ]:
# from nltk.corpus import stopwords
# from pymystem3 import Mystem
# from string import punctuation

# Перевод в леммы работал очень медленно,
# поэтому эти импорты закомменчены…

> ### Шаг 1. Откройте таблицу и изучите общую информацию о данных

In [ ]:
payment_stat = pd.read_csv('./datasets/data.csv')
display(payment_stat.info())

In [ ]:
# до изменений в данных проверяю что у категорий пенсионер и безработный
# рабочий стаж указан в часах
display(
    payment_stat.loc[(payment_stat['income_type'] == 'безработный') \
            | (payment_stat['income_type'] == 'пенсионер'), 'days_employed'].describe()
)

In [ ]:
display(payment_stat.head())

In [ ]:
# проверим явные дубликаты
display(len(payment_stat.loc[payment_stat.duplicated()]))

In [ ]:
# удалим явные дубликаты
payment_stat.drop_duplicates(inplace=True)

И чтобы никаких больше дубликатов!

In [ ]:
# проверим явные дубликаты
display(len(payment_stat.loc[payment_stat.duplicated()]))

##### columns
children, days_employed, dob_years,

education, education_id, family_status, family_status_id,

gender, income_type, debt, total_income, purpose

In [ ]:
# оченим разброс значений по столбцу 'days_employed'
display(payment_stat['days_employed'].describe())

In [ ]:
# оченим разброс значений по столбцу 'total_income'
display(payment_stat['total_income'].describe())

> ### Шаг 2. Предобработка данных

#### Шаг 2.1. Пропуски
> В двух столбцах есть пропущенные значения, найдите их и заполните медианным значением по столбцу:

> *  опишите, какие пропущенные значения вы обнаружили;

<div id="caps_01"></div>

`days_employed` и `total_income` равны по количеству ненулевых значений.

> Похоже, кто не работал, тот и не заработал.

    © Кэп

> *  проверьте, какую долю составляют пропущенные значения в каждом из столбцов с пропусками;

In [ ]:
days_employed_na = payment_stat['days_employed'].isna().sum()
total_income_na = payment_stat['total_income'].isna().sum()

# казалось, что считать isna следует с помощью count(),
# такой подсчет возвращает 0…

numer_of_rows = payment_stat.shape[0]
print('Количество пропксков:')
print('\t\'days_employed\' =', days_employed_na, end=' ')
print('\tэто{percent: .1%}'.format(percent=days_employed_na/numer_of_rows))
print('\t\'total_income\' =', total_income_na, end=' ')
print('\tэто{percent: .1%}'.format(percent=total_income_na/numer_of_rows))

> *  приведите возможные причины появления пропусков в данных;

Во всех 54 явных дубликатах были `NaN` в `days_employed` и `total_income`.

А дальше давайте разбираться!

In [ ]:
# уникальные значения в столбце 'income_type' 
display(payment_stat['income_type'].unique())

In [ ]:
display(payment_stat.loc[((payment_stat['days_employed'].isna())
                 | (payment_stat['total_income'].isna()))
                 & (payment_stat['income_type'] == 'безработный')
                 ].shape)

In [ ]:
display(payment_stat.loc[((payment_stat['days_employed'].isna())
                 | (payment_stat['total_income'].isna()))
                 & (payment_stat['income_type'] == 'студент')
                 ].shape)

`Студенты` и `безработные` не "скрыли" свой доход или рабочие дни…

Кэп был неправ [здесь](#caps_01).

In [ ]:
display(payment_stat.loc[(payment_stat['days_employed'].isna())
                 & (payment_stat['total_income'].isna())
                 ]['income_type'].unique()
       )

Пропуски только в этих категориях.

In [ ]:
display(payment_stat['income_type'].unique())

In [ ]:
display(payment_stat.loc[(payment_stat['days_employed'].isna())
                 & (payment_stat['total_income'].isna())
                 ].head()
       )

In [ ]:
display(
    payment_stat['education'].unique(),
    len(payment_stat['education'].unique())
)

payment_stat['education'] = payment_stat['education'].str.lower()

display(
    payment_stat['education'].unique(),
    len(payment_stat['education'].unique())
)

In [ ]:
display(
    payment_stat['education_id'].unique(),
    len(payment_stat['education_id'].unique())
)

In [ ]:
display(
    payment_stat['purpose'].unique(),
    len(payment_stat['purpose'].unique())
)

Ну никакого порядка!

Ещё вернёмся к этому изобилию…

In [ ]:
# для красоты причешу
payment_stat['family_status'] = payment_stat['family_status'].str.lower()
display(payment_stat['family_status'].unique())

In [ ]:
display(payment_stat['gender'].unique())

Это что за зверь **XNA**?

In [ ]:
display(payment_stat.groupby(by='gender')['gender'].count().sort_values(ascending=False))
display(payment_stat.loc[payment_stat['gender'] == 'XNA'])

In [ ]:
# Пусть же XNA будет девушкой, пожалуйста…
payment_stat.loc[payment_stat['gender'] == 'XNA', 'gender'] = 'F'

In [ ]:
display(payment_stat.groupby(by='gender')['gender'].count().sort_values(ascending=False))

In [ ]:
payment_stat.loc[(payment_stat['days_employed'].isna())
                 & (payment_stat['total_income'].isna())
                 ].groupby(by='education')['education'].count().sort_values(ascending=False)

In [ ]:
payment_stat.loc[(payment_stat['days_employed'].isna())
                 & (payment_stat['total_income'].isna())
                 ].groupby(by='gender')['gender'].count().sort_values(ascending=False)

In [ ]:
payment_stat.loc[(payment_stat['days_employed'].isna())
                 & (payment_stat['total_income'].isna())
                 ].groupby(by='income_type')['income_type'].count().sort_values(ascending=False)

In [ ]:
display(payment_stat['income_type'].unique())

In [ ]:
payment_stat.loc[(payment_stat['days_employed'].isna())
                 & (payment_stat['total_income'].isna())
                 ].groupby(by='dob_years')['dob_years'].count().sort_values(ascending=False).head()

In [ ]:
payment_stat.loc[(payment_stat['days_employed'].isna())
                 & (payment_stat['total_income'].isna())
                 ].groupby(by='children')['children'].count().sort_values(ascending=False)

-1 — это значит ребёнок в проекте?

In [ ]:
payment_stat.loc[payment_stat['children'] == -1, 'children'] = 1

In [ ]:
display(
    payment_stat['children'].describe()
)

In [ ]:
display(
    payment_stat.groupby(by='children')['children'].count().sort_values(ascending=False)
)

In [ ]:
payment_stat.loc[payment_stat['children'] == 20].head()

Двадцать детей — это класс!

Но не 76 раз)

Выглядит малоправдоподобно и разрыв между 5 и 20 никак не представлен.

In [ ]:
payment_stat.loc[payment_stat['children'] == 20, 'children'] = 2

In [ ]:
payment_stat.loc[(payment_stat['days_employed'].isna())
                 & (payment_stat['total_income'].isna())
                 ].groupby(by='debt')['debt'].count().sort_values(ascending=False)

Закончим модуль по статистике, посчитаю корреляцию)

На первый взгляд закономерность появления `NaN` не видна…

> *  объясните, почему заполнить пропуски медианным значением — лучшее решение для количественных переменных.

`Медиана` при выбросах будет точнее описывать "среднее" значение. То есть `медиана` менее чувствительна к выбросам в данных, чем `среднее арифметическое`.

#### Шаг 2.2. Аномалии

In [ ]:
# тут хорошо бы уточнить откуда минус в данных
# возьму минусовые значения по модулю

payment_stat.loc[payment_stat['days_employed'] < 0, 'days_employed'] \
    = payment_stat.loc[payment_stat['days_employed'] < 0, 'days_employed'].abs()

display(payment_stat['days_employed'].describe())

Да, можно было и весь столбец «замодулить», но будем беречь вычеслительные мощности)

Не знаю как это сделать не так громоздко… 🤷‍♂️

In [ ]:
to_plot_incom_by_days = payment_stat.groupby(by='dob_years')['days_employed'].median()
display(to_plot_incom_by_days.head())
sns.scatterplot(x=to_plot_incom_by_days.index, y=to_plot_incom_by_days.array)

In [ ]:
to_plot_incom_by_days = payment_stat.groupby(by='income_type')['days_employed'].median()
display(to_plot_incom_by_days)
sns.scatterplot(x=to_plot_incom_by_days.array, y=to_plot_incom_by_days.index)

In [ ]:
# предположим что если трудовой стаж болше 100 лет или 36 500 дней,
# то он указан в часах. Переведем в дни
payment_stat.loc[payment_stat['days_employed'] > 36_500, 'days_employed'] \
    = payment_stat.loc[payment_stat['days_employed'] > 36_500, 'days_employed'] / 24

display(payment_stat['days_employed'].describe())

In [ ]:
# посмотрим максимальный стаж в годах
print('Да я в Data Science отпахал', round(payment_stat['days_employed'].max()/365, 1), 'лет!')

In [ ]:
# что со столбцом возраст заёмщика?
display(payment_stat['dob_years'].describe())

Рождённый с кредитом!

In [ ]:
# сколько их таких с нулевым возрастом?
display(payment_stat.loc[payment_stat['dob_years'] == 0, 'dob_years'].count())

Далматин?

_Пора ввести отдельный курс по предобработке данных и готовить первокласных предобработчиков. Чтобы больше никаких нулей…_

In [ ]:
display(
    payment_stat.loc[payment_stat['dob_years'] == 0] \
        .groupby(by='education')['education'].count()
)

In [ ]:
hi_edu_age_median = payment_stat.loc[(payment_stat['dob_years'] != 0)
                                     & (payment_stat['education'] == 'высшее'), 'dob_years'].median()
cont_edu_age_median = payment_stat.loc[(payment_stat['dob_years'] != 0)
                                     & (payment_stat['education'] == 'неоконченное высшее'), 'dob_years'].median()
mid_edu_age_median = payment_stat.loc[(payment_stat['dob_years'] != 0)
                                     & (payment_stat['education'] == 'среднее'), 'dob_years'].median()

print('высшее образование, медиана возраста:', hi_edu_age_median)
print('неоконченное высшее образование, медиана возраста:', cont_edu_age_median)
print('среднее образование, медиана возраста:', mid_edu_age_median)

In [ ]:
payment_stat.loc[(payment_stat['dob_years'] == 0)
    & (payment_stat['education'] == 'высшее'), 'dob_years'] = int(hi_edu_age_median)
payment_stat.loc[(payment_stat['dob_years'] == 0)
    & (payment_stat['education'] == 'неоконченное высшее'), 'dob_years'] = int(cont_edu_age_median)
payment_stat.loc[(payment_stat['dob_years'] == 0)
    & (payment_stat['education'] == 'среднее'), 'dob_years'] = int(mid_edu_age_median)

display(payment_stat.loc[payment_stat['dob_years'] == 0]['dob_years'].count())

In [ ]:
# что теперь со столбцом возраст заёмщика?
display(payment_stat['dob_years'].describe())

In [ ]:
# меняю NaN на медианное значение
median_of_days_employed = payment_stat['days_employed'].median()
payment_stat.fillna(value=median_of_days_employed, inplace=True)

median_of_dob_years = payment_stat['total_income'].median()
payment_stat.fillna(value=median_of_dob_years, inplace=True)

##### columns
children, days_employed, dob_years,

education, education_id, family_status, family_status_id,

gender, income_type, debt, total_income, purpose

#### Шаг 2.3. Тип данных

In [ ]:
payment_stat['total_income'] = payment_stat['total_income'].round(decimals=0)
payment_stat['total_income'] = payment_stat['total_income'].astype('int')

In [ ]:
display(payment_stat.info())

#### Шаг 2.4. Строки-дубликаты

In [ ]:
# после всех этих усреднений и на дубликаты проверить не грех
display(len(payment_stat.loc[payment_stat.duplicated()]))

In [ ]:
payment_stat.drop_duplicates(inplace=True, ignore_index=True)

> - поясните, как выбирали метод для поиска и удаления дубликатов в данных;

         - стандартными `.duplicated()` и `.drop_duplicates()`
         - вручную при помощи `.unique()` и `.str.lower()` 

> - приведите возможные причины появления дубликатов;

         - человек который вводил данные ошибся 👽
         - после того как заменили `NaN` медианными значениям 👌
         - ошибки программ | компьютеров на этапе сбора и записи данных 🤖

#### Шаг 2.5. Два новых датафрейма

In [ ]:
education_dict = pd.DataFrame(columns=['education_id', 'education'])
family_dict = pd.DataFrame(columns=['family_status_id', 'family_status'])

education_dict['education_id'] = payment_stat['education_id'].unique()
education_dict['education'] = education_dict['education_id']\
    .apply(lambda x: payment_stat.loc[payment_stat['education_id'] == x, 'education'].unique()[0])

family_dict['family_status_id'] = payment_stat['family_status_id'].unique()
family_dict['family_status'] = family_dict['family_status_id']\
    .apply(lambda x: payment_stat.loc[payment_stat['family_status_id'] == x, 'family_status'].unique()[0])

display(education_dict.head())
display(family_dict.head())

In [ ]:
payment_stat.drop(labels=['education', 'family_status'], axis='columns', inplace=True)
#display(payment_stat.head())
display(payment_stat.tail())

#### Шаг 2.6. Категории дохода

In [ ]:
def set_category(income_value):
    if 0 <= income_value <= 30000:
        return 'E'
    elif 30001 <= income_value <= 50000:
        return 'D'
    elif 50001 <= income_value <= 200000:
        return 'C'
    elif 200001 <= income_value <= 1000000:
        return 'B'
    elif 1000001 <= income_value:
        return 'A'

In [ ]:
print(set_category(50000))

In [ ]:
payment_stat['total_income_category'] = payment_stat['total_income'].apply(set_category)
display(payment_stat.head())

#### Шаг 2.7. Категории целей кредита

In [ ]:
# 'операции с автомобилем'
# 'операции с недвижимостью'
# 'проведение свадьбы'
# 'получение образования'
#Create lemmatizer and stopwords list

# на kaggle нашел функцию, выглядит логично и прилично
# но работает очень долго…
# https://www.kaggle.com/alxmamaev/how-to-easy-preprocess-russian-text

# mystem = Mystem() 
# russian_stopwords = stopwords.words("russian")

# def purpose_categoriser(purpose_str):
#     tokens = mystem.lemmatize(purpose_str.lower())
#     tokens = [token for token in tokens if token not in russian_stopwords\
#               and token != " " \
#               and token.strip() not in punctuation]
    
#     purpose_str = " ".join(tokens)
#     print(purpose_str)
#     if 'автомобиль' in purpose_str:
#         return 'операции с автомобилем'
#     elif ('жилье' in purpose_str) | ('недвижимость' in purpose_str):
#         return 'операции с недвижимостью'
#     elif 'свадьба' in purpose_str:
#         return 'проведение свадьбы'
#     elif 'образование' in purpose_str:
#         return 'получение образования'

In [ ]:
# Жаль что через леммы очень долго…

def purpose_categoriser_if(purpose_string):
    purpose_string = purpose_string.lower()
    if (
        'автомобиль' in purpose_string or 'автомобиля' in purpose_string
        or 'автомобили' in purpose_string or 'автомобилем' in purpose_string
    ):
         return 'операции с автомобилем'
    elif (
        'недвижимость' in purpose_string or 'недвижимости' in purpose_string
        or 'недвижимостью' in purpose_string or 'жильем' in purpose_string
        or 'жилье' in purpose_string or 'жилью' in purpose_string
        or 'жилья' in purpose_string
    ):
         return 'операции с недвижимостью'
    elif (
        'свадьба' in purpose_string or 'свадьбу' in purpose_string
        or 'свадьбы' in purpose_string
    ):
         return 'проведение свадьбы'
    elif (
        'образованием' in purpose_string or 'образования' in purpose_string
        or 'образование' in purpose_string
    ):
         return 'получение образования'

In [ ]:
#display(
#    payment_stat['purpose'].unique(),
#    len(payment_stat['purpose'].unique())
#)

#whole_purposes = payment_stat['purpose'].unique()
payment_stat['purpose_category'] = payment_stat.loc[: , 'purpose'].apply(purpose_categoriser_if)

#display(payment_stat['purpose_category'].unique())

In [ ]:
display(payment_stat['purpose_category'])

In [ ]:
display(payment_stat['purpose_category'].unique())

### Шаг 3. Ответы на вопросы

> Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [ ]:
payment_stat.groupby(['children', 'debt'])['debt'].count()

> Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [ ]:
payment_stat.groupby(['family_status_id', 'debt'])['debt'].count()

> Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [ ]:
payment_stat.groupby(['total_income_category', 'debt'])['debt'].count()

> Как разные цели кредита влияют на его возврат в срок?

In [ ]:
debt_by_pourpouse_category = payment_stat.groupby(['purpose_category', 'debt'])['purpose'].count()
display(debt_by_pourpouse_category)
display(debt_by_pourpouse_category.iloc[: , 0])

In [ ]:
debt_by_pourpouse_category = payment_stat.pivot_table(values='purpose',
                                                      index='purpose_category',
                                                      columns='debt',
                                                      aggfunc='count', sort=True)
display(debt_by_pourpouse_category)

In [ ]:
sns.histplot(
    debt_by_pourpouse_category,
    y="purpose_category", hue='debt',
    x='purpose',
    multiple="stack",
    palette="light:m_r",
    edgecolor=".3",
    linewidth=.5,
)

In [ ]:
# оченим разброс значений по столбцу 'total_income'
display(payment_stat['total_income'].describe())